In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
from transformers import ViTForImageClassification, SwinForImageClassification
from transformers import AutoFeatureExtractor
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import torch

import os
import pandas as pd
from PIL import Image
import numpy as np

import glob

import torch.nn as nn
import torch
import torch.nn.functional as F
import math
import torchvision

from functools import partial
from torchvision.models.detection import FasterRCNN

import sys
sys.path.append('/hdd/yuchen/pipeline/training_pipeline')
from models.layers import (
    Backbone, 
    PatchEmbed, 
    Block, 
    get_abs_pos,
    get_norm,
    Conv2d,
    LastLevelMaxPool
)
from models.utils import _assert_strides_are_log2_contiguous


from torchvision.models.detection.faster_rcnn import FastRCNNConvFCHead
from torchvision.models.detection.rpn import AnchorGenerator, RPNHead
from torchvision.ops.feature_pyramid_network import LastLevelMaxPool

sys.path.append('/hdd/yuchen/pipeline/training_pipeline/models')

from faster_rcnn_swin_transformer_detection import transforms
from faster_rcnn_swin_transformer_detection.network_files import BackboneWithFPN
from faster_rcnn_swin_transformer_detection.swin_transformer import swin_t, Swin_T_Weights, swin_b, Swin_B_Weights

## MAE to teacher

In [49]:
ckpt = torch.load('/hdd/yuchen/satdata/weights/fmow_pretrain.pth')['model']

teacher_model.vit.embeddings.patch_embeddings.projection.weight = torch.nn.Parameter(ckpt['patch_embed.proj.weight'])
teacher_model.vit.embeddings.patch_embeddings.projection.bias = torch.nn.Parameter(ckpt['patch_embed.proj.bias'])

for layer_num in range(24):
    
    teacher_model.vit.encoder.layer[layer_num].attention.attention.query.weight = torch.nn.Parameter(ckpt[f'blocks.{layer_num}.attn.qkv.weight'][0:1024])
    teacher_model.vit.encoder.layer[layer_num].attention.attention.query.bias = torch.nn.Parameter(ckpt[f'blocks.{layer_num}.attn.qkv.bias'][0:1024])
    teacher_model.vit.encoder.layer[layer_num].attention.attention.key.weight = torch.nn.Parameter(ckpt[f'blocks.{layer_num}.attn.qkv.weight'][1024:2048])
    teacher_model.vit.encoder.layer[layer_num].attention.attention.key.bias = torch.nn.Parameter(ckpt[f'blocks.{layer_num}.attn.qkv.bias'][1024:2048])
    teacher_model.vit.encoder.layer[layer_num].attention.attention.value.weight = torch.nn.Parameter(ckpt[f'blocks.{layer_num}.attn.qkv.weight'][2048:])
    teacher_model.vit.encoder.layer[layer_num].attention.attention.value.bias = torch.nn.Parameter(ckpt[f'blocks.{layer_num}.attn.qkv.bias'][2048:])
    
    teacher_model.vit.encoder.layer[layer_num].intermediate.dense.weight = torch.nn.Parameter(ckpt[f'blocks.{layer_num}.mlp.fc1.weight'])
    teacher_model.vit.encoder.layer[layer_num].intermediate.dense.bias = torch.nn.Parameter(ckpt[f'blocks.{layer_num}.mlp.fc1.bias'])
    teacher_model.vit.encoder.layer[layer_num].output.dense.weight = torch.nn.Parameter(ckpt[f'blocks.{layer_num}.mlp.fc2.weight'])
    teacher_model.vit.encoder.layer[layer_num].output.dense.bias = torch.nn.Parameter(ckpt[f'blocks.{layer_num}.mlp.fc2.bias'])

In [52]:
del ckpt

## MAE to student

In [2]:
ckpt = torch.load('/hdd/yuchen/satdata/weights/swinmae100.pth')

In [14]:
student_model = SwinForImageClassification.from_pretrained("microsoft/swin-tiny-patch4-window7-224")
student_model.classifier = nn.Linear(in_features=768, out_features=2, bias=True)

In [49]:
idx = 0
ckpt[f'layers.{idx}.blocks.0.attn.qkv.weight'].shape

torch.Size([288, 96])

In [46]:
ckpt = torch.load('/hdd/yuchen/satdata/weights/swinmae100.pth')

student_model.swin.embeddings.patch_embeddings.projection.weight = torch.nn.Parameter(ckpt['patch_embed.proj.weight'])
student_model.swin.embeddings.patch_embeddings.projection.bias = torch.nn.Parameter(ckpt['patch_embed.proj.bias'])
student_model.swin.embeddings.norm.weight = torch.nn.Parameter(ckpt['patch_embed.norm.weight'])
student_model.swin.embeddings.norm.bias = torch.nn.Parameter(ckpt['patch_embed.norm.bias'])

for idx in [0,1,2,3]:
    student_model.swin.encoder.layers[idx].blocks[0].layernorm_before.weight = torch.nn.Parameter(ckpt[f'layers.{idx}.blocks.0.norm1.weight'])
    student_model.swin.encoder.layers[idx].blocks[0].layernorm_before.bias = torch.nn.Parameter(ckpt[f'layers.{idx}.blocks.0.norm1.bias'])

    curr_size = student_model.swin.encoder.layers[idx].blocks[0].attention.self.query.weight.shape[0]
    
    student_model.swin.encoder.layers[idx].blocks[0].attention.self.query.weight = torch.nn.Parameter(ckpt[f'layers.{idx}.blocks.0.attn.qkv.weight'][:curr_size])
    student_model.swin.encoder.layers[idx].blocks[0].attention.self.key.weight = torch.nn.Parameter(ckpt[f'layers.{idx}.blocks.0.attn.qkv.weight'][curr_size:curr_size*2])
    student_model.swin.encoder.layers[idx].blocks[0].attention.self.value.weight = torch.nn.Parameter(ckpt[f'layers.{idx}.blocks.0.attn.qkv.weight'][curr_size*2:])
    student_model.swin.encoder.layers[idx].blocks[0].attention.self.query.bias = torch.nn.Parameter(ckpt[f'layers.{idx}.blocks.0.attn.qkv.bias'][:curr_size])
    student_model.swin.encoder.layers[idx].blocks[0].attention.self.key.bias = torch.nn.Parameter(ckpt[f'layers.{idx}.blocks.0.attn.qkv.bias'][curr_size:curr_size*2])
    student_model.swin.encoder.layers[idx].blocks[0].attention.self.value.bias = torch.nn.Parameter(ckpt[f'layers.{idx}.blocks.0.attn.qkv.bias'][curr_size*2:])
    student_model.swin.encoder.layers[idx].blocks[0].attention.output.dense.weight = torch.nn.Parameter(ckpt[f'layers.{idx}.blocks.0.attn.proj.weight'])
    student_model.swin.encoder.layers[idx].blocks[0].attention.output.dense.bias = torch.nn.Parameter(ckpt[f'layers.{idx}.blocks.0.attn.proj.bias'])
    
    student_model.swin.encoder.layers[idx].blocks[0].layernorm_after.weight = torch.nn.Parameter(ckpt[f'layers.{idx}.blocks.0.norm2.weight'])
    student_model.swin.encoder.layers[idx].blocks[0].layernorm_after.bias = torch.nn.Parameter(ckpt[f'layers.{idx}.blocks.0.norm2.bias'])
    
    student_model.swin.encoder.layers[idx].blocks[0].intermediate.dense.weight = torch.nn.Parameter(ckpt[f'layers.{idx}.blocks.0.mlp.fc1.weight'])
    student_model.swin.encoder.layers[idx].blocks[0].intermediate.dense.bias = torch.nn.Parameter(ckpt[f'layers.{idx}.blocks.0.mlp.fc1.bias'])
    
    student_model.swin.encoder.layers[idx].blocks[0].output.dense.weight = torch.nn.Parameter(ckpt[f'layers.{idx}.blocks.0.mlp.fc2.weight'])
    student_model.swin.encoder.layers[idx].blocks[0].output.dense.bias = torch.nn.Parameter(ckpt[f'layers.{idx}.blocks.0.mlp.fc2.bias'])
    
    
    student_model.swin.encoder.layers[idx].blocks[1].layernorm_before.weight = torch.nn.Parameter(ckpt[f'layers.{idx}.blocks.1.norm1.weight'])
    student_model.swin.encoder.layers[idx].blocks[1].layernorm_before.bias = torch.nn.Parameter(ckpt[f'layers.{idx}.blocks.1.norm1.bias'])
    
    student_model.swin.encoder.layers[idx].blocks[1].attention.self.query.weight = torch.nn.Parameter(ckpt[f'layers.{idx}.blocks.1.attn.qkv.weight'][:curr_size])
    student_model.swin.encoder.layers[idx].blocks[1].attention.self.key.weight = torch.nn.Parameter(ckpt[f'layers.{idx}.blocks.1.attn.qkv.weight'][curr_size:curr_size*2])
    student_model.swin.encoder.layers[idx].blocks[1].attention.self.value.weight = torch.nn.Parameter(ckpt[f'layers.{idx}.blocks.1.attn.qkv.weight'][curr_size*2:])
    student_model.swin.encoder.layers[idx].blocks[1].attention.self.query.bias = torch.nn.Parameter(ckpt[f'layers.{idx}.blocks.1.attn.qkv.bias'][:curr_size])
    student_model.swin.encoder.layers[idx].blocks[1].attention.self.key.bias = torch.nn.Parameter(ckpt[f'layers.{idx}.blocks.1.attn.qkv.bias'][curr_size:curr_size*2])
    student_model.swin.encoder.layers[idx].blocks[1].attention.self.value.bias = torch.nn.Parameter(ckpt[f'layers.{idx}.blocks.1.attn.qkv.bias'][curr_size*2:])
    student_model.swin.encoder.layers[idx].blocks[1].attention.output.dense.weight = torch.nn.Parameter(ckpt[f'layers.{idx}.blocks.1.attn.proj.weight'])
    student_model.swin.encoder.layers[idx].blocks[1].attention.output.dense.bias = torch.nn.Parameter(ckpt[f'layers.{idx}.blocks.1.attn.proj.bias'])
    
    student_model.swin.encoder.layers[idx].blocks[1].layernorm_after.weight = torch.nn.Parameter(ckpt[f'layers.{idx}.blocks.1.norm2.weight'])
    student_model.swin.encoder.layers[idx].blocks[1].layernorm_after.bias = torch.nn.Parameter(ckpt[f'layers.{idx}.blocks.1.norm2.bias'])
    student_model.swin.encoder.layers[idx].blocks[1].intermediate.dense.weight = torch.nn.Parameter(ckpt[f'layers.{idx}.blocks.1.mlp.fc1.weight'])
    student_model.swin.encoder.layers[idx].blocks[1].intermediate.dense.bias = torch.nn.Parameter(ckpt[f'layers.{idx}.blocks.1.mlp.fc1.bias'])
    student_model.swin.encoder.layers[idx].blocks[1].output.dense.weight = torch.nn.Parameter(ckpt[f'layers.{idx}.blocks.1.mlp.fc2.weight'])
    student_model.swin.encoder.layers[idx].blocks[1].output.dense.bias = torch.nn.Parameter(ckpt[f'layers.{idx}.blocks.1.mlp.fc2.bias'])
    
    if idx != 3:
        student_model.swin.encoder.layers[idx].downsample.reduction.weight =  torch.nn.Parameter(ckpt[f'layers.{idx}.downsample.reduction.weight'])
        student_model.swin.encoder.layers[idx].downsample.norm.weight =  torch.nn.Parameter(ckpt[f'layers.{idx}.downsample.norm.weight'])
        student_model.swin.encoder.layers[idx].downsample.norm.bias =  torch.nn.Parameter(ckpt[f'layers.{idx}.downsample.norm.bias'])

In [47]:
student_model

SwinForImageClassification(
  (swin): SwinModel(
    (embeddings): SwinEmbeddings(
      (patch_embeddings): SwinPatchEmbeddings(
        (projection): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
      )
      (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): SwinEncoder(
      (layers): ModuleList(
        (0): SwinStage(
          (blocks): ModuleList(
            (0-1): 2 x SwinLayer(
              (layernorm_before): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
              (attention): SwinAttention(
                (self): SwinSelfAttention(
                  (query): Linear(in_features=96, out_features=96, bias=True)
                  (key): Linear(in_features=96, out_features=96, bias=True)
                  (value): Linear(in_features=96, out_features=96, bias=True)
                  (dropout): Dropout(p=0.0, inplace=False)
                )
                (output): SwinSelfOutput(
  

## Student to MAE

In [135]:
backbone = swin_t(weights=Swin_T_Weights.DEFAULT).features

In [137]:
backbone[1][0].attn.qkv.weight.shape

torch.Size([288, 96])

In [146]:
swin = 0
torch.nn.Parameter(torch.stack([ckpt[f'swin.encoder.layers.{swin}.blocks.0.attention.self.query.weight'],
                                     ckpt[f'swin.encoder.layers.{swin}.blocks.0.attention.self.key.weight'],
                                    ckpt[f'swin.encoder.layers.{swin}.blocks.0.attention.self.value.weight']]).flatten(0,1)).shape

torch.Size([288, 96])

In [133]:
backbone[0][0].weight = torch.nn.Parameter(ckpt['swin.embeddings.patch_embeddings.projection.weight'])
backbone[0][0].bias = torch.nn.Parameter(ckpt['swin.embeddings.patch_embeddings.projection.bias'])
backbone[0][2].weight = torch.nn.Parameter(ckpt['swin.embeddings.norm.weight'])
backbone[0][2].bias = torch.nn.Parameter(ckpt['swin.embeddings.norm.bias'])

for idx in [(1,0),(3,1),(5,2),(7,3)]:
    b = idx[0]
    swin = idx[1]
    backbone[b][0].norm1.weight = torch.nn.Parameter(ckpt[f'swin.encoder.layers.{swin}.blocks.0.layernorm_before.weight'])
    backbone[b][0].norm1.bias = torch.nn.Parameter(ckpt[f'swin.encoder.layers.{swin}.blocks.0.layernorm_before.bias'])
    
    backbone[b][0].attn.qkv.weight =  torch.nn.Parameter(torch.stack([ckpt[f'swin.encoder.layers.{swin}.blocks.0.attention.self.query.weight'],
                                     ckpt[f'swin.encoder.layers.{swin}.blocks.0.attention.self.key.weight'],
                                    ckpt[f'swin.encoder.layers.{swin}.blocks.0.attention.self.value.weight']]).flatten(0,1))
    
    backbone[b][0].attn.qkv.bias =  torch.nn.Parameter(torch.stack([ckpt[f'swin.encoder.layers.{swin}.blocks.0.attention.self.query.bias'],
                                     ckpt[f'swin.encoder.layers.{swin}.blocks.0.attention.self.key.bias'],
                                    ckpt[f'swin.encoder.layers.{swin}.blocks.0.attention.self.value.bias']]).flatten(0,1))
    
    backbone[b][0].attn.proj.weight = torch.nn.Parameter(ckpt[f'swin.encoder.layers.{swin}.blocks.0.attention.output.dense.weight'])
    backbone[b][0].attn.proj.bias = torch.nn.Parameter(ckpt[f'swin.encoder.layers.{swin}.blocks.0.attention.output.dense.bias'])
    
    backbone[b][0].norm2.weight = torch.nn.Parameter(ckpt[f'swin.encoder.layers.{swin}.blocks.0.layernorm_after.weight'])
    backbone[b][0].norm2.bias = torch.nn.Parameter(ckpt[f'swin.encoder.layers.{swin}.blocks.0.layernorm_after.bias'])
    
    backbone[b][0].mlp[0].weight = torch.nn.Parameter(ckpt[f'swin.encoder.layers.{swin}.blocks.0.intermediate.dense.weight'])
    backbone[b][0].mlp[0].bias = torch.nn.Parameter(ckpt[f'swin.encoder.layers.{swin}.blocks.0.intermediate.dense.bias'])
    backbone[b][0].mlp[3].weight = torch.nn.Parameter(ckpt[f'swin.encoder.layers.{swin}.blocks.0.output.dense.weight'])
    backbone[b][0].mlp[3].bias = torch.nn.Parameter(ckpt[f'swin.encoder.layers.{swin}.blocks.0.output.dense.bias'])
    
    backbone[b][1].norm1.weight = torch.nn.Parameter(ckpt[f'swin.encoder.layers.{swin}.blocks.1.layernorm_before.weight'])
    backbone[b][1].norm1.bias = torch.nn.Parameter(ckpt[f'swin.encoder.layers.{swin}.blocks.1.layernorm_before.bias'])
    
    backbone[b][1].attn.qkv.weight =  torch.nn.Parameter(torch.stack([ckpt[f'swin.encoder.layers.{swin}.blocks.1.attention.self.query.weight'],
                                     ckpt[f'swin.encoder.layers.{swin}.blocks.1.attention.self.key.weight'],
                                    ckpt[f'swin.encoder.layers.{swin}.blocks.1.attention.self.value.weight']]).flatten(0,1))
    
    backbone[b][1].attn.qkv.bias =  torch.nn.Parameter(torch.stack([ckpt[f'swin.encoder.layers.{swin}.blocks.1.attention.self.query.bias'],
                                     ckpt[f'swin.encoder.layers.{swin}.blocks.1.attention.self.key.bias'],
                                    ckpt[f'swin.encoder.layers.{swin}.blocks.1.attention.self.value.bias']]).flatten(0,1))
    
    backbone[b][1].attn.proj.weight = torch.nn.Parameter(ckpt[f'swin.encoder.layers.{swin}.blocks.1.attention.output.dense.weight'])
    backbone[b][1].attn.proj.bias = torch.nn.Parameter(ckpt[f'swin.encoder.layers.{swin}.blocks.1.attention.output.dense.bias'])

    backbone[b][1].norm2.weight = torch.nn.Parameter(ckpt[f'swin.encoder.layers.{swin}.blocks.1.layernorm_after.weight'])
    backbone[b][1].norm2.bias = torch.nn.Parameter(ckpt[f'swin.encoder.layers.{swin}.blocks.1.layernorm_after.bias'])
    
    backbone[b][1].mlp[0].weight = torch.nn.Parameter(ckpt[f'swin.encoder.layers.{swin}.blocks.1.intermediate.dense.weight'])
    backbone[b][1].mlp[0].bias = torch.nn.Parameter(ckpt[f'swin.encoder.layers.{swin}.blocks.1.intermediate.dense.bias'])
    backbone[b][1].mlp[3].weight = torch.nn.Parameter(ckpt[f'swin.encoder.layers.{swin}.blocks.1.output.dense.weight'])
    backbone[b][1].mlp[3].bias = torch.nn.Parameter(ckpt[f'swin.encoder.layers.{swin}.blocks.1.output.dense.bias'])
    
    if b != 7:
        backbone[b+1].reduction.weight = torch.nn.Parameter(ckpt[f'swin.encoder.layers.{swin}.downsample.reduction.weight'])
        backbone[b+1].norm.weight = torch.nn.Parameter(ckpt[f'swin.encoder.layers.{swin}.downsample.norm.weight'])
        backbone[b+1].norm.bias = torch.nn.Parameter(ckpt[f'swin.encoder.layers.{swin}.downsample.norm.bias'])

## backup code

In [ ]:
# class ViT(Backbone):
#     """
#     This module implements Vision Transformer (ViT) backbone in :paper:`vitdet`.
#     "Exploring Plain Vision Transformer Backbones for Object Detection",
#     https://arxiv.org/abs/2203.16527
#     """

#     def __init__(
#         self,
#         img_size=1024,
#         patch_size=16,
#         in_chans=3,
#         embed_dim=768,
#         depth=12,
#         num_heads=12,
#         mlp_ratio=4.0,
#         qkv_bias=True,
#         drop_path_rate=0.0,
#         norm_layer=nn.LayerNorm,
#         act_layer=nn.GELU,
#         use_abs_pos=True,
#         use_rel_pos=False,
#         rel_pos_zero_init=True,
#         window_size=0,
#         window_block_indexes=(),
#         residual_block_indexes=(),
#         use_act_checkpoint=False,
#         pretrain_img_size=224,
#         pretrain_use_cls_token=True,
#         out_feature="last_feat",
#     ):
  
#         super().__init__()
#         self.pretrain_use_cls_token = pretrain_use_cls_token

#         self.patch_embed = PatchEmbed(
#             kernel_size=(patch_size, patch_size),
#             stride=(patch_size, patch_size),
#             in_chans=in_chans,
#             embed_dim=embed_dim,
#         )

#         if use_abs_pos:
#             # Initialize absolute positional embedding with pretrain image size.
#             num_patches = (pretrain_img_size // patch_size) * (pretrain_img_size // patch_size)
#             num_positions = (num_patches + 1) if pretrain_use_cls_token else num_patches
#             self.pos_embed = nn.Parameter(torch.zeros(1, num_positions, embed_dim))
#         else:
#             self.pos_embed = None

#         # stochastic depth decay rule
#         dpr = [x.item() for x in torch.linspace(0, drop_path_rate, depth)]

#         self.blocks = nn.ModuleList()
#         for i in range(depth):
#             block = Block(
#                 dim=embed_dim,
#                 num_heads=num_heads,
#                 mlp_ratio=mlp_ratio,
#                 qkv_bias=qkv_bias,
#                 drop_path=dpr[i],
#                 norm_layer=norm_layer,
#                 act_layer=act_layer,
#                 use_rel_pos=use_rel_pos,
#                 rel_pos_zero_init=rel_pos_zero_init,
#                 window_size=window_size if i in window_block_indexes else 0,
#                 use_residual_block=i in residual_block_indexes,
#                 input_size=(img_size // patch_size, img_size // patch_size),
#             )
#             if use_act_checkpoint:
#                 # TODO: use torch.utils.checkpoint
#                 from fairscale.nn.checkpoint import checkpoint_wrapper

#                 block = checkpoint_wrapper(block)
#             self.blocks.append(block)

#         self._out_feature_channels = {out_feature: embed_dim}
#         self._out_feature_strides = {out_feature: patch_size}
#         self._out_features = [out_feature]

#         if self.pos_embed is not None:
#             nn.init.trunc_normal_(self.pos_embed, std=0.02)

#         self.apply(self._init_weights)

#     def _init_weights(self, m):
#         if isinstance(m, nn.Linear):
#             nn.init.trunc_normal_(m.weight, std=0.02)
#             if isinstance(m, nn.Linear) and m.bias is not None:
#                 nn.init.constant_(m.bias, 0)
#         elif isinstance(m, nn.LayerNorm):
#             nn.init.constant_(m.bias, 0)
#             nn.init.constant_(m.weight, 1.0)

#     def forward(self, x):
#         x = self.patch_embed(x)
#         if self.pos_embed is not None:
#             x = x + get_abs_pos(
#                 self.pos_embed, self.pretrain_use_cls_token, (x.shape[1], x.shape[2])
#             )

#         for blk in self.blocks:
#             x = blk(x)

#         outputs = {self._out_features[0]: x.permute(0, 3, 1, 2)}
#         return outputs

      
# num_classes = 2
# embed_dim, depth, num_heads, dp = 768, 12, 12, 0.1
    
# embed_dim=1024
# depth=24
# num_heads=16
    
# mlp_ratio=4
# img_size = 224
# patch_size = 16
# window_size = 14
# mlp_ratio = 4
    
# # Load the pretrained SqueezeNet1_1 backbone.
# net = ViT(  # Single-scale ViT backbone
#         img_size=img_size,
#         patch_size=patch_size,
#         embed_dim=embed_dim,
#         depth=depth,
#         num_heads=num_heads,
#         drop_path_rate=dp,
#         window_size=14,
#         mlp_ratio=4,
#         qkv_bias=True,
#         norm_layer=partial(nn.LayerNorm, eps=1e-6),
#         window_block_indexes=[
#             # 2, 5, 8 11 for global attention
#             0,
#             1,
#             3,
#             4,
#             6,
#             7,
#             9,
#             10,
#         ],
#         residual_block_indexes=[],
#         use_rel_pos=True,
#         out_feature="last_feat",
#     )


# # ckpt = torch.utis('weights/mae_pretrain_vit_base.pth')
# # ckpt = torch.hub.load_state_dict_from_url('https://dl.fbaipublicfiles.com/mae/pretrain/mae_pretrain_vit_base.pth')
# # ckpt = torch.hub.load_state_dict_from_url('https://dl.fbaipublicfiles.com/mae/pretrain/mae_pretrain_vit_large.pth')
# # ckpt = torch.load('/hdd/yuchen/satdata/weights/fmow_pretrain.pth')

# # # from safetensors.torch import save_file, load_file
# # # loaded = load_file("/hdd/yuchen/satdata/weights/checkpoint-175200/model.safetensors")
# # # net.load_state_dict(loaded, strict=False)
# # # ckpt = torch.load('/hdd/yuchen/satdata/weights/scalemae-vitlarge-800.pth')
# # net.load_state_dict(ckpt['model'], strict=False)
